In [7]:
%load_ext autoreload
%autoreload 2
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/src")
import helpers as hlp

In [9]:
import numpy as np
import pandas as pd

# AWS EC2

Instructions for setup: https://github.com/GalvanizeDataScience/lectures/blob/SF/aws/aws_lecture.ipynb

Keep python script running: https://stackoverflow.com/questions/47823240/keep-running-a-python-script-on-aws-ec2-even-if-cli-session-is-closed

## tripadvisor_web_scraping_ec2

```
Copying Files to EC2
To copy a file myfile.txt to EC2, use a command like this.
scp myfile.txt my_first_ec2:
To copy a file from the EC2 to the current directory on the local machine, try
scp my_first_ec2:path/to/remote_file .
To copy a directory mydir to EC2, use a command like this.
scp -r mydir my_first ec2:
```

### Run web scraper
```
ssh tripadvisor_web_scraping_ec2

tmux new -s mywindow

```
``` python

python3 src/web_scraping_aws_test.py

#OR

python3 src/web_scraping_aws.py

```
```

tmux a -t mywindow

tmux kill-window -t mywindow

```

### Copy csv files from ec2
zip -r aws.zip aws

scp tripadvisor_web_scraping_ec2:data/web_scraped/aws.zip .

# Create Pipeline with Test Files

In [41]:
aws_test_data_dir = '../data/web_scraped/aws_test/'

## Load & Merge CSV Data for All Hotels

In [44]:
# Get all folders - 1 per tripadvisor hotel result page
folders = os.listdir(aws_test_data_dir)
folders

['pg1', 'pg4', 'pg3', 'pg2']

In [45]:
# From each folder, read all CSV files - 1 per hotel
dfs = []
for folder in folders:
#     print(folder)
    data_dir = aws_test_data_dir + '/' + folder + '/'
    csv_filenames = os.listdir(data_dir)    
    for name in csv_filenames:
        dfs.append(pd.read_csv(data_dir + name))
df_all_hotels = pd.concat(dfs, ignore_index=True)
df_all_hotels.shape

(266, 8)

In [47]:
df_all_hotels.head()

,review_id,url,hotel_name,review_date,review_body,user_location,helpful_vote,rating
0,750305270,https://www.tripadvisor.com/Hotel_Review-g6098...,OHANA Waikiki Malia by Outrigger,"March 10, 2020",If you are visiting Honolulu on a budget this ...,"Medicine Hat, Canada",0,3.0
1,750121846,https://www.tripadvisor.com/Hotel_Review-g6098...,OHANA Waikiki Malia by Outrigger,"March 9, 2020",2 nights here. Bed was comfortable shower pres...,NaN,0,4.0
2,746226779,https://www.tripadvisor.com/Hotel_Review-g6098...,OHANA Waikiki Malia by Outrigger,"February 19, 2020",The property is in a decent location with cour...,"Las Vegas, Nevada",1,1.0
3,752651761,https://www.tripadvisor.ca/Hotel_Review-g60982...,Hilton Hawaiian Village Waikiki Beach Resort,"April 15, 2020",I enjoyed my stay overall.The room itself was ...,"Seattle, Washington",35,4.0
4,752585766,https://www.tripadvisor.ca/Hotel_Review-g60982...,Hilton Hawaiian Village Waikiki Beach Resort,"April 13, 2020",Our family stayed here as a last second switch...,"Pella, Iowa",3,5.0


In [50]:
df_all_hotels.tail()

,review_id,url,hotel_name,review_date,review_body,user_location,helpful_vote,rating
261,748360164,https://www.tripadvisor.com/Hotel_Review-g6098...,Aqua Aloha Surf Waikiki,"February 29, 2020","Check in staff (and check-out) were very, very...","Island of Hawaii, Hawaii",34,2.0
262,737718477,https://www.tripadvisor.com/Hotel_Review-g6098...,Aqua Aloha Surf Waikiki,"January 7, 2020",Located within walking distance from the beach...,NaN,3,3.0
263,752329491,https://www.tripadvisor.com/Hotel_Review-g6098...,Airport Honolulu Hotel,"April 6, 2020","The hotel is nice and clean, even during the C...",NaN,0,5.0
264,751890692,https://www.tripadvisor.com/Hotel_Review-g6098...,Airport Honolulu Hotel,"March 27, 2020",The hotel was great. Clean room and is dated a...,"Mudgee, Australia",0,5.0
265,750263150,https://www.tripadvisor.com/Hotel_Review-g6098...,Airport Honolulu Hotel,"March 10, 2020",We just needed an overnight place to sleep. Th...,"Laguna Beach, California",4,2.0


In [48]:
df_all_hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 8 columns):
review_id        266 non-null int64
url              266 non-null object
hotel_name       266 non-null object
review_date      266 non-null object
review_body      266 non-null object
user_location    175 non-null object
helpful_vote     266 non-null int64
rating           266 non-null float64
dtypes: float64(1), int64(2), object(5)
memory usage: 16.8+ KB


In [49]:
df_all_hotels.describe()

,review_id,helpful_vote,rating
count,2.660000e+02,266.000000,266.000000
mean,7.474683e+08,18.706767,4.109023
std,1.283151e+07,47.475162,1.246638
min,6.453464e+08,0.000000,1.000000
25%,7.489957e+08,0.000000,4.000000
50%,7.513375e+08,1.000000,5.000000
75%,7.519708e+08,11.750000,5.000000
max,7.534573e+08,325.000000,5.000000


In [52]:
df_all_hotels['review_body'][0]

'If you are visiting Honolulu on a budget this hotel is good... although not on Waikiki beach it is within a 10 minute walk. But this is a “budget” hotel. No bells and whistles... pool is not really much of anything... decor is dated (think 1980s Holiday Inn). The staff is friendly but the cleaning staff unfortunately tended to be smokers so after my room was cleaned I would come in to that smell and would have to open the patio doors to air it out so I just started putting the do not disturb on the door so they wouldn’t clean.'

# Appendix

# AWS S3 Bucket

In [10]:
import boto3

s3 = boto3.client('s3')

In [11]:
for b in s3.list_buckets()['Buckets']:
    print(b['Name'])

chels-first-bucket-2020
chels-second-bucket-2020
pair-bucket-2020-chels-helnaz
pair-bucket-2020-chels-helnaz-2nd
spark-rdds-ind
tripadvisor-hotel-reviews


In [12]:
response = s3.list_objects_v2(Bucket='tripadvisor-hotel-reviews')
for obj in response['Contents']:
    print(obj['Key'])

test.txt


In [ ]:
# Download file

# s3.download_file(Bucket=bucket_name,
#                  Key=remote_file_name,
#                  Filename="downloaded-" + local_file_name)